# **Torrent Downloader**
**Important Note:** Set the runtime in GPU to give more space and fast speed in downloading.

> Go to Runtime -> Change Runtime and give GPU as the Hardware Accelerator.  You will get around 384GB to download any torrent you want.

# [1] **Install all necessary modules**
This will initiate the downloads.



[(fixed) Error on downloading libtorrent](https://stackoverflow.com/questions/66362159/modulenotfounderror-no-module-named-libtorrent)



In [ ]:
!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent
!apt install python3-libtorrent
import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

#[2] **Mount the drive**
In order to stream or download files, please mount your own personal google drive.

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


# [3] **Get your Torrent file or Magnet Links**
Use the cell below to upload your `.torrent` file or copy the whole magnet link in your friendly torrent site.

# .Torrent File
Upload your `.torrent` file here

In [ ]:


from google.colab import files

source = files.upload()
params = {
    "save_path": "/content/drive/My Drive/Torrent",
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))

# Magnet Links
Paste your magnet links here, after that input `q` to stop this cell 

- you can add many links as much as you can =) 

In [ ]:
params = {"save_path": "/content/drive/My Drive/Torrent"}
n=0
while True:
    n+=1
    magnet_link = input(f"[Link {n}] - Enter Magnet Link Or Type q to Exit: ")
    if magnet_link.lower() == "q":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )

# [4] **Time to Download!**
Click the cell to start the download. While downloading do your other task =) 

In [ ]:
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "Download Complete, check your drive")
    time.sleep(1)

#**Stream my Gdrive files on VLC**

**in order to stream the file, set it to anyone with the link.**

*Prerequisities:*
- Your Google Drive file link [Copy the id only! after the /d/<id>] 
- Your same google drive account api key get from google cloud

*How to use this:*
- 1.) Paste the video from your google drive *make sure set it to public*

- 2.) Paste the Drive API key created from [google cloud console](https://console.cloud.google.com/)

In [ ]:
import re

def gdrive_link_filter(gdrive_id):
  string1 = re.sub(r"https:\/\/drive\.google\.com\/file\/d\/", "", gdrive_id)
  string2 = re.sub(r"\/view\?usp=share_link", "", string1)
  return string2
  
while True:
  gdrive_file = str(input('Please input the Google Drive file you want to stream [copy the id only]: '))
  if gdrive_file !='':
    api_key = str(input('Please input you api key from the Google Cloud: '))
    if api_key !='':
      media = f'https://www.googleapis.com/drive/v3/files/{gdrive_link_filter(gdrive_file)}?alt=media&key={api_key}'
      print("here's your streamble link: doesnt work? check your file id?")
      print('='*20)
      print(media)
      break
    

